### NLP vs Text Mining
- NLP : 인간의 언어를 이해하는데 집중된 학문(음성인식, AI비서, 챗봇)
- Text Mining : 비정형 텍스트데이터에서 의미 있는 정보를 추출하는 학문

### Text Mining 종류
- 텍스트 분류 : 텍스트가 속하는 카테고리를 분류하는 분석
- 감성분석 : 텍스트에서 나타나는 감정/기분/의견 등을 분석
- 텍스트 요약 : 텍스트에서 중요한 주제를 나타내는 키워드, 문장 등을 추출/생성
- 텍스트 군집화와 유사도측정 : 텍스트의 비슷한 정도를 측정하고 그룹핑하는 분석

### 텍스트 분석 프로세스
1. 텍스트 전처리
    - 클렌징(대/소문자 변경, 특수문자 삭제)
    - 토큰화 : 텍스트를 잘게 쪼개는 작업
    - Stop word(불용어) 제거
    - 어근추출(Stemming/Lemmatization)
        - 영어 : nltk
        - 한국어 : konlpy(형태소분석기)
2. 피처 벡터화/추출 : 정형화 작업, 글자에서 정해진 알고리즘에 따라 숫자로 변경
   - 단순 언어 빈도
   - BOW(Bag of Word)
   - tf-idf
   - word embedding(워드임베딩)
   - BPE(바이트페어인코딩)
3. ML 학습/예측/평가

### 데이터 로딩

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 100)   # 컬럼의 너비를 조정해서 긴 리뷰를 볼 수 있음

In [ ]:
train = pd.read_csv("./data/ratings_train.txt", delimiter="\t")
test = pd.read_csv("./data/ratings_test.txt", delimiter="\t")

In [ ]:
display("Train data")
display(train.head())
display("Test data")
display(test.head())

#### 결측치 확인

In [ ]:
display(train.info())
display(test.info())

#### 결측치 데이터 삭제

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
train.shape, test.shape

### 텍스트 분석시 사용하는 단어
- 말뭉치(corpus) : 텍스트분석을 위해 모은 데이터셋
- 문서(document) : 말뭉치 안에 텍스트를 지칭하는 단위

### 단어빈도 분석
- 워드카운트 : 단어의 등장 빈도를 측정하는 알고리즘

In [ ]:
text_train = train['document']

In [ ]:
text_train

In [ ]:
textList=[]
text_train[0].split().append(textList)
textList

In [ ]:
# 토큰화
# 띄어쓰기 단위로 모든 문장을 쪼개서 리스트에 넣어보자.

tmp = [doc.split(" ") for doc in text_train]
token_list = []
for t in tmp:
    token_list += t

In [ ]:
token_list

In [ ]:
import itertools
text_train_list=[]
for i in text_train:
    i = i.split(" ")
    text_train_list.append(i)
token_list = list(itertools.chain(*text_train_list))
token_list

In [ ]:
from collections import Counter
counter = Counter(token_list)
counter.most_common(40)

In [ ]:
# 토큰화 ver2
token_list = [t for doc in text_train for t in doc.split(" ")]

#### 워드클라우드
- 단어의 빈도를 기반으로 크기를 조절해 시각화하는 방법

In [ ]:
# !pip install wordcloud

In [ ]:
from wordcloud import WordCloud  # 클래스

In [ ]:
wc = WordCloud(background_color="white", font_path="C:\Windows\Fonts\malgunsl.ttf", random_state=802)

In [ ]:
reviews = " ".join(token_list)   # 하나의 문자열로 변경

In [ ]:
cloud = wc.generate_from_text(reviews)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(cloud)
plt.axis("off")  # x축,y축 끄기
plt.savefig("./naver_movie_wordcloud.jpg")   # 그림저장
plt.show()

### 단어 유사도 분석
- gensim : word2vec, topic modeling 등 텍스트마이닝 알고리즘이 구현된 라이브러리
- word2vec : 구글에서 발표한 단어 사이의 관계를 수치화하는 알고리즘

In [ ]:
# !pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2c = Word2Vec(window=5,          # 중심단어를 기준으로 주변에 몇개단어까지 관계성을 학습할지
               min_count=5,       # 전체 말뭉치의 최소 등장 빈도 수 설정(최소빈도에 도달하지 못하면 학습단어로 사용X)
               sg=1,              # 중심단어를 y, 주변단어를 x로 설정하는 알고리즘(skip-gram)
               vector_size=100,   # 한 개의 단어를 몇 개의 숫자로 표현할건지
                                  # 숫자가 커질수록 각 단어의 관계를 풍부하게 표현가능
               sentences=[doc.split(" ") for doc in text_train])

In [ ]:
w2c.wv.get_vector("영화")    # 영화를 표현하는 숫자(100개 숫자)

In [ ]:
w2c.wv.most_similar("원빈")

### 감성분석
- 감성사전을 이용한 분석
- 머신러닝을 이용한 분석
   - 1. 토큰화 진행
   - 2. BOW / tf-idf를 이용해 수치화(정형화)

#### BOW
- 문맥의 순서를 무시하고 단어의 빈도를 이용해 수치화하는 방법
- 문맥의 순서를 무시하기 때문에 자연어처리에서는 부적합하다.
- 하지만 일반 텍스트마이닝에서는 충분히 활용 가치가 있다.

#### BOW 진행 순서
1. 단어사전 구축
2. 단어사전을 기반으로 각 문장에서 해당 단어가 등장하는 빈도를 측정

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv_test = CountVectorizer()

In [ ]:
# 단어사전 구축
# 영어 -> 대문자를 소문자로 변경, 1글자 제외, 띄어쓰기를 기준으로 토큰화
# 한글 -> 띄어쓰기를 기준으로 토큰화
cv_test.fit(text_train)

In [ ]:
len(cv_test.vocabulary_) # 등록된 단어의 갯수 확인

In [ ]:
temp = ["안녕 나는 뽀로로야.",
        "오늘 점심은 뽀로로야.",
        "안녕 나는 오늘 점심에 뽀로로를 먹을꺼야.",
        "너는 오늘 나와 집에 같이 가자."]

In [ ]:
cv_test.fit(temp)

In [ ]:
display(len(cv_test.vocabulary_))
display(cv_test.vocabulary_)

In [ ]:
# 단어빈도를 측정한 수치화
cv_test.transform(temp)

In [ ]:
cv_test.transform(temp).toarray()

#### 학습 데이터 구성

In [ ]:
naver_movie_cv = CountVectorizer()
naver_movie_cv.fit(text_train)  # 단어사전 구축
X_train = naver_movie_cv.transform(text_train)  # 빈도기반의 수치화
X_test = naver_movie_cv.transform(test['document'])

In [ ]:
y_train = train['label']  # 0:부정, 1:긍정
y_test = test['label']

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#### 모델링

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
logi_model = LogisticRegression()

In [ ]:
rs = cross_val_score(logi_model, X_train, y_train, cv=3)

In [ ]:
rs

In [ ]:
# 학습 및 예측
logi_model.fit(X_train,y_train)
sample_reviews = ["이 영화는 이제껏 보여줬고 보여줘야했던 토르의 인생이 얼마나 굴곡지고 외롭고 가슴 아픈지를 아주 가벼운 연출로 보여주면서 그 슬픔을 희석 시켜줍니다. 만약 이 내용을 아주 진지하게 그려냈다면 보다가 중간에 포기했을 것 같아요. 제인과 토르의 이야기도 발키리의 개인 서사도 잘 다뤄주면서 동시에 토르가 처한 상황과 그가 잃은 것들이 얼마나 많고 그게 얼마나 슬픈 것인지도 보여줍니다. 그러면서 마지막엔 희망을 이야기하기까지 합니다. 히어로물이라는 장르를 사랑한 이유가 바로 이 마지막때문인데 그런 면에서 아주 좋았습니다.",
                  "스토리 전개가 그냥 이상하고 다 끊기는 느낌 이거 재밌다는 사람들 부럽다 온세상이 재밌을 듯 토르3 때문에 믿었는데 올해 최고의 믿는 도끼였다", 
                  "베일 미쳤다... 배우 하나로 영화 퀄이 달라지네", 
                  "잤음 쿨쿨 ... 신나게잤음",
                  "쓸데없이 고퀄인 애들용 만화 .. 본 것 같음 마블의 시대가 저물어 가겠구나"]
sample_transform = naver_movie_cv.transform(sample_reviews) # 정형화 작업
pre = logi_model.predict_proba(sample_transform)  # 확률값 예측

In [ ]:
pre

#### 학습된 단어(토큰)별 가중치 확인

In [ ]:
naver_coef = logi_model.coef_[0]
len(naver_coef)

In [ ]:
naver_vocab = naver_movie_cv.vocabulary_
len(naver_vocab)

In [ ]:
print(naver_coef[0])
naver_vocab

In [ ]:
# vocab 정렬하기
import pandas as pd
df = pd.DataFrame([naver_vocab.keys(), naver_vocab.values()], index=["단어","인덱스"]).T.sort_values(by="인덱스").set_index("인덱스")
# vacab과 가중치 결합
df['가중치'] = naver_coef
df

In [ ]:
# 가중치를 중심으로 정렬 -> 상위 30, 하위 30개 단어 추출
df.sort_values(by="가중치", inplace=True, ascending=False)
top30 = df.head(30)
bottom30 = df.tail(30)

In [ ]:
top30

In [ ]:
bottom30

In [ ]:
# 한글로 출력하기 위해서 폰트 지정하기
from matplotlib import rc
rc('font', family = 'Malgun Gothic')

# 마이너스 나오게 하기
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

# 시각화
main_words = pd.concat([top30, bottom30])
main_words.set_index("단어").plot(kind='bar', figsize=(20,5), rot=70)

- 08/03

#### tf-idf
- tf : 개별문서에서의 특정단어 빈도
    - 개별문서에서 특정단어의 빈도가 높으면 해당문서를 대표하는 특징이 된다.
- df : 특정단어를 가지고 있는 문서의 개수
    - 전체 말뭉치에서 너무 과도하게 등장하면 특징으로서 의미가 없어진다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

##### 하이퍼파라미터 튜닝
- 파이프라인을 이용해 텍스트 전처리와 모델링 2단계로 이루어진 작업을 병합하자.
- TfidVectorizer 하이퍼파라미터
   - max_df : 특정단어가 말뭉치에서 등장하는 최대 횟수 지정
   - min_df : 특정단어가 말뭉치에서 등장하는 최소 횟수 지정
   - max_features : feature로 최대 사용되는 단어 수를 지정
   - stop_words : 분석에서 사용하지 않는 단어를 지정
   - n_gram_range : 토큰을 묶어서 feature로 활용하는 정도를 지정
- LogisticRegression 하이퍼파라미터
   - C : 규제 파라미터 -> 값이 커지면 규제가 약한, 값이 작아지면 규제가 커진다.

In [ ]:
# 파이프라인 : 머신러닝 작업을 하나의 프로세스로 묶어주는 것
# ex) 결측치 -> 스케일러 -> 인코더 -> 모델

from sklearn.pipeline import Pipeline

In [ ]:
# 파이프라인 생성
pipeline = Pipeline([
    ('movie_tfidf', TfidfVectorizer()),
    ('movie_logi', LogisticRegression())
])

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_param = {
    "movie_tfidf__max_df" : [7000,8500,10000],
    "movie_tfidf__min_df" : [1,5,8,10],
    "movie_tfidf__ngram_range" : [(1,2),(2,2),(1,3)],
    "movie_logi__C" : [0.001,0.01,0.1,10,100,1000]
}

In [ ]:
X_train

In [ ]:
grid = GridSearchCV(pipeline, grid_param, cv=3, n_jobs=-1)
grid.fit(text_train,y_train)    # 날것의 데이터 text_train

In [ ]:
rs.mean()

In [ ]:
# 결과확인
print("최고점수 : ",grid.best_score_)
print("최고조합 : ",grid.best_params_)
# 최적의 파라미터로 학습된 모델
best_model = grid.best_estimator_
# 단어사전 수
print(len(best_model.steps[0][1].vocabulary_))

In [ ]:
best_model.predict(["이 영화는 너무 재밌다.", "정말 내 인생 최악의 영화다."])

### 한국어를 위한 Konlpy 사용하기

In [4]:
from konlpy.tag import Kkma
from konlpy.tag import Okt

In [6]:
kkm = Kkma()
okt = Okt()

In [3]:
kkm.morphs("아버지가 방에 들어가신다.") # 형태소 단위로 추출

['아버지', '가', '방', '에', '들어가', '시', 'ㄴ다', '.']

In [7]:
okt.morphs("아버지가 방에 들어가신다")

['아버지', '가', '방', '에', '들어가신다']

#### 품사 추출

In [8]:
kkm.pos("아버지가 방에 들어가신다.")

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

In [9]:
kkm.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [11]:
okt.pos("아버지가 방에 들어가신다.")

[('아버지', 'Noun'),
 ('가', 'Josa'),
 ('방', 'Noun'),
 ('에', 'Josa'),
 ('들어가신다', 'Verb'),
 ('.', 'Punctuation')]

In [10]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

#### sklearn과 연결하기

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [31]:
# 사용자정의 tokenizer 함수 정의
def myTokenizer(text):
    rs = okt.pos(text)  # 품사 부착
    df = pd.DataFrame(rs, columns=["형태소", "품사"])
    # 불리언 색인
    df2 = df[(df['품사']=='Noun') | (df['품사']=='Adjective')]
    return df2["형태소"]

In [36]:
# tokenizer 파라미터 설정
tfidf = TfidfVectorizer(tokenizer=myTokenizer, stop_words=["아버지","수박"])

In [33]:
sample_text = ["아버지가 방에 들어가신다.",
               "아름다운 밤하늘에 별이 반짝반짝 빛나고 있다.",
               "시원한 계곡 소리를 들으며 맛있는 수박을 먹고 싶다."
              ]

In [37]:
tfidf.fit(sample_text)   # 단어사전 구축

C:\Users\AI\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(stop_words=['아버지', '수박'],
                tokenizer=<function myTokenizer at 0x0000014B89645D30>)

In [38]:
tfidf.vocabulary_

{'방': 3,
 '아름다운': 7,
 '밤하늘': 2,
 '별': 4,
 '있다': 8,
 '시원한': 6,
 '계곡': 0,
 '소리': 5,
 '맛있는': 1}